---
title: "Basic Network Analysis in R"
author: 
  - name: David Schoch
    orcid: 0000-0003-1689-0557
    email: david.schoch@gesis.org
    url: https://mr.schochastics.net
    affiliations:
      - name: GESIS - Leibniz Institute for the Social Sciences
toc: true
format:
  html: default
  ipynb: default
execute:
  echo: true
  warning: false
license: CC BY-NC
jupyter:
  kernelspec:
    name: "ir"
    language: "R"
    display_name: "R 4.4.1"
---

# Learning Objectives

By the end of this tutorial, you will 

1. have a practical understand of network analysis
2. be able to work confidently with the R package `igraph`
3, be able to analyze a diverse set of different types of networks

# Target audience

This tutorial is aimed at beginners in network analysis with knowledge in R.

# Setting up the computational environment

To run all the code in this tutorial, you need to install and load three packages.

In [ ]:
install.packages(c("igraph", "netrankr"))
devtools::install_github("schochastics/networkdata")

`igraph` is used for the majority of analytic tasks and for its data structures. `netrankr` is a package
for network centrality. 

In [ ]:
library(igraph)
library(netrankr)
library(networkdata)

In [ ]:
library(ggraph)
library(graphlayouts)

Make sure you have at least the version given below. Some of the examples may not be backward compatible.

In [ ]:
packageVersion("igraph")
packageVersion("netrankr")
packageVersion("networkdata")

# Social Science Usecase(s)

Network analysis provides researchers with a versatile framework for examining relationships and structures within complex social systems. 
For example, in studying the dynamics of an online support community, one could use network analysis to explore how individuals connect, 
share resources, and offer support. By analyzing metrics such as degree centrality (identifying key individuals who connect others), 
clustering coefficients (revealing tightly-knit groups), and betweenness centrality (spotlighting users who bridge subgroups), researchers 
can understand how information and support flow within the community. This approach can uncover subgroups that form around 
shared needs, as well as highlight key figures who facilitate support or knowledge sharing. 
Network analysis also enables researchers to track changes in the community over time, such as the emergence of new
leaders or the diffusion of helpful resources, offering a better understanding of social cohesion, influence, and resilience 
within the community.

# Introduction

The main focus of this tutorial is empirical analysis of networks and skips a lot of additional functionality of igraph
For the most part of this tutorial, we assume that the network data is already present in R.
Reading in data may not pose much of an issue if you are already familiar with R, but can be quite a
challenge if you are new to R. The last section of this tutorial is devoted to this topic in great detail. 

The tutorial does introduce key terms for network analysis, but much of the theory behind them is not explained in great detail. This tutorial is meant to be "hands-on", giving practical help for empirical 
work. If you are interested in the technical/theoretical details behind certain methods, please consult 
relevant literature (a list is given at the end of the tutorial).


# Which package to choose?

Well, the last section spoiled already what we are going to use in this tutorial, but for the 
sake of completeness, I will discuss some other packages here and motivate why I would recommend 
`igraph` as the goto package for standard network analytic tasks.

Besides methods, `igraph` also provides data structures which facilitate to store and process network data. Two other packages 
that allow for this are `graph` and `network`. The former is, however, not available on CRAN anymore, only via
Bioconductor. The latter provides the foundation for much of the statistical modelling aspects for networks such as
Exponential Random Graph Models (ERGMs) and Stochastic Actor Oriented Models (SAOMs).

The figure below shows how many packages on CRAN rely on those three packages (i.e. they are mentioned in `Depends`, `Imports`, or `Suggests`).

In [ ]:
sg <- readRDS("crannet.RDS")
ggraph(sg, "stress") +
    geom_edge_link0(
        edge_color = "grey66", edge_width = 0.3,
        arrow = arrow(
            angle = 15, length = unit(0.15, "inches"),
            ends = "last", type = "closed"
        )
    ) +
    geom_node_point(shape = 21, aes(fill = col, size = seed)) +
    scale_fill_brewer(type = "qual", name = "") +
    scale_size_manual(values = c(2, 5), guide = "none") +
    guides(fill = guide_legend(override.aes = list(size = 5))) +
    theme_void() +
    theme(legend.position = "bottom") +
    coord_equal(clip = "off")

The figure was produced with the help of the `cranet` package ([link](https://github.com/mbojan/cranet)).
`igraph` seems to be clearly favored by the R community. So if you install a package for, say, signed network analysis,
changes are high that it depends on the graph structures provided by `igraph`. Besides the data structures,
the package offers a large variety of network analytic methods which are all implemented in C. The methods are well optimized and
also work quite well for large graphs. 

The `network` package historically shares some commonalities with `igraphs` data structures. The package itself, though is really only providing the data structure and no analytic methods. The `sna` package ([link](https://cran.r-project.org/package=sna)) implements network analytic tools using the data structures provided by `network`.
Overall, the syntax and provided methods are very much comparable between `igraph` and `sna` and they are almost interchangeable in this regard. The advantage of igraph is its speed. I have run several benchmark tasks and `igraph` usually comes out on top. That being said, there is no real case to be made against `network`/`sna`. If you are into statistical modelling of networks, then that should actually be the preferred choice since the `ergm` package is build on top of `network`. In this case you probably also
want to look at the meta package `statnet` ([link](http://statnet.org/)) which includes `network`, `sna`, and `ergm` (among other packages).

The package `intergraph` ([link](https://cran.r-project.org/package=intergraph)) can be used if you need to switch representations between `igraph` and `network`.


---

# Basic network notations

Networks are commonly represented with an **adjacency matrix** or via an **edgelist**. If you are interested 
in a "tidy way" checkout my [tidygraph](../tidynetAnaR/) tutorial. Below, we represent friendship relations between Bob, Ann, and Steve as a matrix and an edgelist.

In [ ]:
# adjacency matrix
A <- matrix(c(0,1,1,1,0,1,1,1,0),nrow = 3,ncol = 3,byrow = TRUE)
rownames(A) <- colnames(A) <- c("Bob","Ann","Steve")
A
#edge list
el <- matrix(c("Bob","Ann","Bob","Steve","Ann","Steve"),
             nrow = 3,ncol = 2, byrow = TRUE)
el

The adjacency matrix $A$ is symmetric, meaning that the relations are undirected, i.e. Bob is friends with
Ann and Ann is friends with Bob. In general, $A[i,j]=1$, if there is a relation between $i$ and $j$.
If $A[i,j]=1$ does not imply $A[j,i]=1$ then $A$ defines a directed network.

Once we have defined an edgelist or an adjacency matrix, we can turn them into `igraph` objects
as follows.

In [ ]:
g1 <- graph_from_adjacency_matrix(A, mode = "undirected", diag = FALSE)
g2 <- graph_from_edgelist(el,directed = FALSE)
#g1 and g2 are the same graph so only printing g1
g1

The printed summary shows some general descriptives of the graph.
The string "UN--" in the first line indicates that the network is *U*ndirected (*D* for directed graphs) and has a *N*ame attribute (we named the nodes Bob, Ann, and Steve). The third and forth character are *W*, if there is a edge weight attribute, and *B* if the network is bipartite (there exists a node attribute "type"). The following number indicate the number of nodes and edges.
The second line lists all graph, node and edge variables. Here, we only have a node attribute "name". 

The conversion from edgelist/adjacency matrix into an igraph object is quite straightforward. The only difficulty is setting the parameters correctly (Is the network directed or not?), especially for edgelists where it may not immediately be obvious if the network is directed or not.

In the following, we use a larger network to introduce some terminology.

In [ ]:
data("greys")

In [ ]:
ggraph(greys, "stress", bbox = 10) +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, aes(fill = sex), size = 5, show.legend = FALSE) +
    geom_node_text(aes(label = name), repel = TRUE) +
    scale_fill_manual(values = c("grey66", "#E8813A", "#4D189D")) +
    theme_void()

The "greys" network is part of the `networkdata` package and consists of most characters from the 
show "Grey's Anatomy" and who hooked up with whom.

In [ ]:
greys

---

# Descriptives and more notations

The **density** of a network is defined as the fraction of the potential edges in a network that are actually present.

In [ ]:
c(
    graph.density(graph.empty(10)),
    graph.density(greys),
    graph.density(graph.full(10))
)

The density of an empty network is $0$ and for the full network it is $1$. The density of empirical network 
lies somewhere in between but as the number of nodes increases, we'd expect the density to decrease and the network 
becomes quite sparse.

A **shortest path** is a path that connects two nodes in a network with a minimal number of edges. The length of a shortest path is called the **distance** between two nodes.

In [ ]:
shortest_paths(greys, from = "Alex Karev", to = "Owen Hunt", output = "vpath")

In [ ]:
E(greys)$epath <- FALSE
E(greys)$epath[as.integer(shortest_paths(greys, from = "Alex Karev", to = "Owen Hunt", output = "epath")$epath[[1]])] <- TRUE

ggraph(greys, "stress", bbox = 10) +
    geom_edge_link0(aes(color = epath, width = epath), show.legend = FALSE) +
    geom_node_point(shape = 21, aes(fill = sex), size = 5, show.legend = FALSE) +
    geom_node_text(aes(label = name), repel = TRUE) +
    scale_fill_manual(values = c("grey66", "#E8813A", "#4D189D")) +
    scale_edge_color_manual(values = c("grey66", "firebrick3")) +
    scale_edge_width_manual(values = c(0.5, 1.5)) +
    theme_void()

In [ ]:
distances(greys)[1:10, 1:10]

The Grey's Anatomy network is **disconnected** and consists of $4$ **connected components**. There are no 
shortest paths between components, which means that the distance is not measurable and set to infinity.

The length of the longest shortest path is called the **diameter** of the network.

In [ ]:
diameter(greys)

In [ ]:
E(greys)$epath <- FALSE
E(greys)$epath[as.integer(shortest_paths(greys, from = "Finn Dandridge", to = "Colin Marlow", output = "epath")$epath[[1]])] <- TRUE

ggraph(greys, "stress", bbox = 10) +
    geom_edge_link0(aes(color = epath, width = epath), show.legend = FALSE) +
    geom_node_point(shape = 21, aes(fill = sex), size = 5, show.legend = FALSE) +
    geom_node_text(aes(label = name), repel = TRUE) +
    scale_fill_manual(values = c("grey66", "#E8813A", "#4D189D")) +
    scale_edge_color_manual(values = c("grey66", "firebrick3")) +
    scale_edge_width_manual(values = c(0.5, 1.5)) +
    theme_void()

**Transitivity** measures the probability that the neighbors of a node are also connected. This is also called the **clustering coefficient**.  

In [ ]:
transitivity(greys, type = "global")
transitivity(greys, type = "local", isolates = "zero")

The global transitivity of an undirected network is the ratio of the triangles and the connected triples in the network.
Local transitivity of a node is the ratio of the triangles connected to the node and the triples centered on the node itself.
In social networks, we generally assume that the transitivity is quite high ("the friend of my friend is also my friend"). In our 
example, we have zero for all values. This is due to the fact that a triangle would require a same sex hook-up which did not occur (*Disclaimer: I never watched the show and gathered the hook ups from various internet resources. So this may well be wrong.*).

For directed networks, a measure of importance is **reciprocity**, which is defined as the proportion of mutual edges between nodes. To illustrate the measure, we use a network of grooming relations among a group of rhesus monkeys.

In [ ]:
data("rhesus")
reciprocity(rhesus)

About `r round(reciprocity(rhesus)*100)`% of edges are reciprocated in the network. The figure below highlights the reciprocated edges. 

In [ ]:
E(rhesus)$mutual <- is.mutual(rhesus)
# plot
ggraph(rhesus, "stress") +
    geom_edge_parallel(aes(filter = !mutual),
        edge_color = "grey66", edge_width = 0.5,
        arrow = arrow(
            angle = 15, length = unit(0.15, "inches"),
            ends = "last", type = "closed"
        ), n = 2, end_cap = circle(8, "pt")
    ) +
    geom_edge_parallel(aes(filter = mutual),
        edge_color = "black", edge_width = 0.5,
        arrow = arrow(
            angle = 15, length = unit(0.15, "inches"),
            ends = "last", type = "closed"
        ), n = 2, end_cap = circle(8, "pt")
    ) +
    geom_node_point(shape = 21, size = 8, aes(fill = gender)) +
    scale_fill_manual(values = c("#E8813A", "#4D189D"), name = "") +
    theme_void() +
    theme(legend.position = "bottom")

## Use case: triad census


In this short use case example, we will discuss the **triad census** of a directed network.
In a directed network, there are 16 possible configurations of edges that can occur between three nodes.

![](triad_census.jpg)

The triad census of a network gives the number of occurrences of each of these triad. Triads are labelled `xyzL` where `x` is the number of reciprocated ties, `y` is the number of unreciprocated ties and `z` is the number of null ties. The `L` term is a letter (U,C,D or T) which allows to differentiate between triads where these numbers are the same. 

One of the many applications of the triad census is to compare a set of networks.
In this example, we are tackling the question of "how transitive is football?" and
asses structural differences among a set of football leagues. 

In [ ]:
data("football_triad")

`football_triad` is a list which contains networks of 112 football leagues as igraph objects. A directed link between
team A and B indicates that A won a match against B. Note that there can also be an edge from B to A,
since most leagues play a double round robin. For the sake of simplicity, all draws were deleted so that
there could also be null ties between two teams if both games ended in a draw.

Below, we calculate the triad census for all network at once using `lapply()`.
The function returns the triad census for each network as a list, which we turn into a matrix
in the second step. Afterwards, we manually add the row and column names of the matrix.

In [ ]:
footy_census <- lapply(football_triad, triad_census)
footy_census <- matrix(unlist(footy_census), ncol = 16, byrow = T)
rownames(footy_census) <- sapply(football_triad, function(x) x$name)
colnames(footy_census) <- c(
    "003", "012", "102", "021D", "021U", "021C", "111D", "111U",
    "030T", "030C", "201", "120D", "120U", "120C", "210", "300"
)

# normalize to make proportions comparable across leagues
footy_census_norm <- footy_census / rowSums(footy_census)

# check the Top 5 leagues
idx <- which(rownames(footy_census) %in% c(
    "england", "spain", "germany",
    "italy", "france"
))
footy_census[idx, ]

Notice how the transitive triad (030T) has the largest count in the top leagues, hinting toward the childhood wisdom:
"If A wins against B and B wins against C, then A must win against C".

In empirical studies, we are not necessarily only interested in transitive triads, but rather how the triad census profiles compare across networks. We follow [Kathrine Faust's](https://doi.org/10.1111%2Fj.1467-9531.2007.00179.x) suggestion and do a singular value decomposition (SVD) on the normalized triad census matrix.

In [ ]:
footy_svd <- svd(footy_census_norm)

SVDs are used to reduce the dimensionality of the data, but retaining most of the information. In our case, the 
data is 16 dimensional, which is impossible to visualize to compare the networks. With an SVD, we can reduce it to two dimensions and get a better visual overview.

In [ ]:
data.frame(
    u1 = footy_svd$d[1] * footy_svd$u[, 1],
    u2 = footy_svd$d[2] * footy_svd$u[, 2],
    league = rownames(footy_census)
) |>
    ggplot(aes(x = u1, y = u2)) +
    geom_point() +
    ggrepel::geom_text_repel(aes(label = league)) +
    theme_minimal() +
    theme(axis.title = element_text(size = 16)) +
    labs(
        x = "First singular vector, multipled by singular value",
        y = "Second singular vector, multipled by singular value"
    )

How to interpret the dimensions? To investigate this question, we take a closer look at the first two dimensions and compare it to some network descriptives. For the sake of brevity, we here only look at the density and proportion of 030T triads. In
general, any node/dyad/triad level statistic could be used.

In [ ]:
data.frame(
    y = footy_svd$d[1] * footy_svd$u[, 1],
    x = sapply(football_triad, graph.density)
) |>
    ggplot(aes(x, y)) +
    geom_point() +
    theme_minimal() +
    theme(axis.title = element_text(size = 16)) +
    labs(x = "density", y = "First singular vector, multipled by singular value")

Density doesn't really seem to be related to the first dimension in this case (in many cases it is!). Might be worthwhile to
explore this further

In [ ]:
data.frame(
    y = footy_svd$d[2] * footy_svd$u[, 2],
    x = footy_census_norm[, 9]
) |>
    ggplot(aes(x, y)) +
    geom_point() +
    theme_minimal() +
    theme(axis.title = element_text(size = 16)) +
    labs(x = "fraction of 030T", y = "Second singular vector, multipled by singular value")

For the second dimension, we get a clearer association. It seems that the fraction of transitive triads 
is a good indicator for structural differences among leagues.

More details can be found in the paper by Kathrine Faust.

---

# Centrality

In a nutshell, a measure of centrality is an index that assigns a numeric values to
the nodes of the network. The higher the value, the more central the node. "Being central" 
is a very ambiguous term and it is thus no surprise that there exists a large variety of 
indices that assess centrality with very different structural properties of the network.

Given the abundance of measures, we will also look at dedicated centrality packages that implement indices
which are not available in `igraph`.

`igraph` contains the following 10 indices:

- degree (`degree()`)
- weighted degree (`graph.strength()`)
- betweenness (`betweenness()`)
- closeness (`closeness()`)
- eigenvector (`eigen_centrality()`)
- alpha centrality (`alpha_centrality()`)
- power centrality (`power_centrality()`)
- PageRank (`page_rank()`)
- eccentricity (`eccentricity()`)
- hubs and authorities (`authority_score()` and `hub_score()`)
- subgraph centrality (`subgraph_centrality()`)

To illustrate some of the indices, we use the "dbces11" graph which is part of the `netrankr` package.

In [ ]:
data("dbces11")

In [ ]:
xy <- graphlayouts::layout_with_stress(dbces11)
ggraph(dbces11, "manual", x = xy[, 1], y = xy[, 2]) +
    geom_edge_link0() +
    geom_node_point(shape = 21, size = 10, fill = "grey66") +
    geom_node_text(aes(label = name)) +
    theme_void() +
    coord_equal(clip = "off")

**degree** simply counts the number of neighbors a node has.

In [ ]:
degree(dbces11)

In [ ]:
id <- which.max(degree(dbces11))
E(dbces11)$deg <- "no"
E(dbces11)$deg[incident(dbces11, 11)] <- "yes"
ggraph(dbces11, "manual", x = xy[, 1], y = xy[, 2]) +
    geom_edge_link(aes(col = deg, width = deg), show.legend = FALSE) +
    geom_node_point(shape = 21, size = 10, fill = "grey66") +
    geom_node_text(aes(label = name)) +
    scale_edge_color_manual(values = c("grey66", "firebrick3")) +
    scale_edge_width_manual(values = c(0.5, 1.5)) +
    theme_void() +
    coord_equal(clip = "off")

**closeness** computes the shortest path distances among nodes. The most central node has the
minimum distance to all other nodes (Since high scores are associated with central nodes, the distances are inverted).

In [ ]:
closeness(dbces11)

The animation below gives an intuition on the calculation for one node.
![](closeness.gif)

**betweeness** is the number of shortest paths that pass through a node (divided by the total number of shortest paths)  

In [ ]:
betweenness(dbces11)

To get an intuition what it means to have a high betweenness, check the network below.

In [ ]:
Kn1 <- graph.full(5)
Kn2 <- graph.full(5)

V(Kn1)$name <- LETTERS[1:5]
V(Kn2)$name <- LETTERS[7:11]

B <- Kn1 %u% Kn2
B <- add.vertices(B, 1, attr = list(name = LETTERS[6]))
B <- add.edges(B, c(5, 11, 6, 11))

ggraph(B, "stress") +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, size = 8, fill = "grey25") +
    geom_node_point(shape = 21, size = 8, aes(filter = name == "F"), fill = "firebrick3") +
    theme_void()

Any shortest path from the right will pass through the red node and vice versa. The 
red note is thus a sort of "gatekeeper" for any information that is passed from left to right.


**eigenvector centrality** extends the idea of degree by assuming that a node is central if it is connected to other central nodes.

In [ ]:
eigen_centrality(dbces11)$vector

**subgraph centrality** is a bit more abstract but what it does is summing up all closed walks weighting them by the 
inverse factorial of its length.

In [ ]:
subgraph_centrality(dbces11)

The remaining indices are mostly designed for directed networks, **page rank** being the prime example. Note, though that
the mentioned indices can also be applied to directed networks. 

If we highlight the most central node for the calculated indices, we get the following. 

In [ ]:
V(dbces11)$cent <- NA
V(dbces11)$cent[which.max(degree(dbces11))] <- "DC"
V(dbces11)$cent[which.max(betweenness(dbces11))] <- "BC"
V(dbces11)$cent[which.max(closeness(dbces11))] <- "CC"
V(dbces11)$cent[which.max(eigen_centrality(dbces11)$vector)] <- "EC"
V(dbces11)$cent[which.max(subgraph_centrality(dbces11))] <- "SC"

ggraph(dbces11, "manual", x = xy[, 1], y = xy[, 2]) +
    geom_edge_link0() +
    geom_node_point(shape = 21, size = 10, aes(fill = cent), show.legend = FALSE) +
    geom_node_text(aes(filter = !is.na(cent), label = cent)) +
    theme_void() +
    coord_equal(clip = "off")

So each index picks a different node as most central. While this is just a toy example, it highlights
how influential the choice of indices can be in empirical settings.

10 is already quite a lot of indices, but there exist far more in the literature. Some of those are
implemented in other packages.

The `sna` package implements roughly the same indices as `igraph` but adds:

- flow betweenness (`flowbet()`)
- load centrality (`loadcent()`)
- Gil-Schmidt Power Index (`gilschmidt()`)
- information centrality (`infocent()`)
- stress centrality (`stresscent()`)

There are also some dedicated centrality packages, such as `centiserve`, `CINNA`, `influenceR` and `keyplayer`.
The biggest in terms of implemented indices is currently `centiserve` with a total of 33 indices.

In [ ]:
library(centiserve)
as.character(lsf.str("package:centiserve"))

The package is maintained by the team behind [centiserver](http://www.centiserver.org/),
the "comprehensive centrality resource and server for centralities calculation".
The website collects indices found in the literature. Currently (February 2022), it lists 403 different indices.
That's...a lot.

The description of `CINNA` says
"Functions for computing, comparing and demonstrating top informative centrality measures within a network."
Most of the indices in the package are imported from other package, such as `centiserve`.
In addition, there are:

- Dangalchev closeness (`dangalchev_closeness_centrality()`)
- group centrality (`group_centrality()`)
- harmonic closeness (`harmonic_centrality()`)
- local bridging centrality (`local_bridging_centrality()`)

The function `calculate_centralities()` can be used to calculate all applicable indices
to a network. The primary purpose of the package is to facilitate the choice of indices
by visual and statistical tools. If you are interested in the details, see this [tutorial](https://www.datacamp.com/community/tutorials/centrality-network-analysis-R)
and this [vignette](https://cran.r-project.org/web/packages/CINNA/vignettes/CINNA.html).

`influenceR` and `keyplayer` are comparably small packages which implement only a small
number of indices.

The choice of indices can be overwhelming and little guidelines exist on when to choose what.
The worst thing to do in any case is to apply a handful of indices and pick the result that suits your
interpretation best. In best case, you have substantive arguments to apply an index and the result 
does match the hypothesis (or not).

## Use case: Florentine Families

A classic example application of centrality indices is the "Florentine Families" dataset, which is included in the `networkdata` package.

In [ ]:
data("flo_marriage")

In [ ]:
ggraph(flo_marriage, "stress") +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, aes(size = wealth), fill = "grey66", show.legend = FALSE) +
    geom_node_text(aes(size = wealth, label = name), show.legend = FALSE) +
    theme_void()

Th network shows marriage ties among Renaissance Families in Florence. Marriages at that time were
strategic to improve the standing of families in society. The size of the names is proportional to the 
wealth of the families. Although the Strozzi were the wealthiest family, it was ultimately the Medici
who became the most powerful family. This is in part due to their central position within this marriage network.

The table bellow shows the ranking for the four most commonly used centrality indices (1=top rank).

In [ ]:
data.frame(
    name = V(flo_marriage)$name,
    degree = rank(-degree(flo_marriage)),
    betweenness = rank(-betweenness(flo_marriage)),
    closeness = rank(-closeness(flo_marriage)),
    eigen = rank(-eigen_centrality(flo_marriage)$vector)
) |>
    knitr::kable(row.names = FALSE)

No matter what structural feature we consider to be important, the Medici always have the most advantageous position.

## Additional Material

I have written a series of blog post about the concept of network centrality, which introduces
some novel tools to assess centrality. These also discuss empirical applications of indices in greater detail. ([1](http://blog.schochastics.net/post/network-centrality-in-r-introduction/),
 [2](http://blog.schochastics.net/post/network-centrality-in-r-neighborhood-inclusion/),
 [3](http://blog.schochastics.net/post/network-centrality-in-r-new-ways-of-measuring-centrality/))

The blog posts rely on the `netrankr` package ([link](http://netrankr.schochastics.net/)), which also 
comes with 9 vignettes that explain the functionality in great detail. Note that the package also
implements around 30 indices, but the index based approach is not its main purpose.


---

# Cliques and Clustering

A *clique* in a network is a set of nodes that form a complete subnetwork within a network (called a complete **subgraph**). A **maximal clique** is a clique that cannot be extended to a bigger clique by addding more nodes to it. 

In [ ]:
data("clique_graph")

All maximal cliques can be calculated with `max_cliques()` (only feasible for fairly small networks). The min parameter can be used to set a minimum size. Here, we want to ignore all cliques of size $2$.

In [ ]:
# only return cliques with three or more nodes
cl <- max_cliques(clique_graph, min = 3)
cl

The figure below shows the network and the found maximal cliques.

In [ ]:
xy <- graphlayouts::layout_with_stress(clique_graph)

cl_df <- as.data.frame(do.call(
    "rbind",
    lapply(seq_along(cl), function(x) {
        cbind(xy[cl[[x]], ], x)
    })
))

ggraph(clique_graph, "stress") +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, size = 8, fill = "grey25") +
    ggforce::geom_mark_hull(data = cl_df, aes(V1, V2, fill = as.factor(x), group = x), show.legend = FALSE) +
    scale_fill_manual(values = c(
        "#E69F00", "#000000", "#56B4E9", "#009E73", "#F0E442", "#0072B2",
        "#D55E00", "#CC79A7", "#666666"
    )) +
    theme_void()

Related to cliques is the **k-core decomposition** of a network. A k-core is a subgraph in which every node has at least k neighbors within the subgraph. A k-core is thus a relaxed version of a clique.  
The function `coreness()` can be used to calculate the k-core membership for each node.

In [ ]:
kcore <- coreness(clique_graph)
kcore

In [ ]:
cl_df <- as.data.frame(do.call(
    "rbind",
    lapply(sort(unique(kcore))[c(2, 3, 4)], function(x) {
        cbind(xy[kcore >= x, ], x)
    })
))

ggraph(clique_graph, "stress") +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, size = 8, fill = "grey25") +
    ggforce::geom_mark_hull(data = cl_df, aes(V1, V2, fill = as.factor(x), group = x), show.legend = FALSE) +
    scale_fill_manual(values = c("red", "blue", "green")) +
    theme_void()

Cliques are the prototypical and most strict definition of a cohesive subgroup of a graph. In empirical networks, however, we rarely encounter situations where we can partition the whole network into a set of 
cliques. A relaxed version of this problem is that of clustering, also referred to as **comunity detection**. 
A cluster is loosely defined as a group of nodes which are internally densely and externally sparsely connected. The network below shows an example for a network with a visible and intuitive cluster structure.

In [ ]:
n1 <- 5
n2 <- 20
set.seed(1234)
g <- sample_islands(n1, n2, 0.9, 5)
g <- simplify(g)
V(g)$grp <- rep(LETTERS[1:n1], each = n2)
ggraph(g, "stress") +
    geom_edge_link0(edge_width = 0.2, edge_color = "grey66") +
    geom_node_point(shape = 21, size = 5, aes(fill = grp), show.legend = FALSE) +
    theme_void()

In contrast, the network below does not really seem to have any well defined cluster structure.

In [ ]:
n1 <- 5
n2 <- 20

set.seed(1234)
g <- sample_islands(n1, n2, 0.25, 15)
g <- simplify(g)
V(g)$grp <- rep(LETTERS[1:n1], each = n2)
ggraph(g, "stress") +
    geom_edge_link0(edge_width = 0.2, edge_color = "grey66") +
    geom_node_point(shape = 21, size = 5, fill = "grey66", show.legend = FALSE) +
    theme_void()

The following algorithms for graph clustering are implemented in `igraph`.

In [ ]:
algs <- as.character(lsf.str("package:igraph"))
algs[stringr::str_detect(algs, "cluster_")]

Most of these algorithms are based on "modularity maximization". Modularity is defined as the fraction of edges that fall within given groups minus the expected fraction if edges were distributed at random.

The workflow of a cluster analysis is always the same, independent from the chosen method. We illustrate the workflow using the infamous karate club network.

In [ ]:
data("karate")

In [ ]:
ggraph(karate, "stress") +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, size = 5, fill = "grey66") +
    theme_void() +
    coord_equal()

In [ ]:
# compute clustering
clu <- cluster_louvain(karate)

# cluster membership vector
mem <- membership(clu)
mem

# clusters as list
com <- communities(clu)
com

To compare the quality of clusterings, we can compute the modularity score for each output.

In [ ]:
imc <- cluster_infomap(karate)
lec <- cluster_leading_eigen(karate)
loc <- cluster_louvain(karate)
sgc <- cluster_spinglass(karate)
wtc <- cluster_walktrap(karate)
scores <- c(
    infomap = modularity(karate, membership(imc)),
    eigen = modularity(karate, membership(lec)),
    louvain = modularity(karate, membership(loc)),
    spinglass = modularity(karate, membership(sgc)),
    walk = modularity(karate, membership(wtc))
)
scores

For the karate network, `cluster_spinglass()` produces the highest modularity score.
The corresponding clustering is shown below.

In [ ]:
V(karate)$clu <- membership(sgc)
ggraph(karate, "stress") +
    geom_edge_link0(edge_color = "grey66") +
    geom_node_point(shape = 21, size = 5, aes(fill = as.factor(clu)), show.legend = FALSE) +
    theme_void() +
    coord_equal()

Modularity maximization is still widely considered as the state-of-the-art clustering method
for networks. There are, however, some technical shortcomings that one should be aware of.
One of those is the so called "resolution limit". When modularity is being maximized, it can happen
that smaller clusters are merged together to form bigger clusters. The prime example is the graph that
consists of cliques connected in a ring.

In [ ]:
n1 <- 5
n2 <- 50
A <- matrix(1, n1, n1)
lst <- vector("list", n2)
lst <- lapply(lst, function(x) A)
AA <- Matrix::bdiag(lst)
for (i in 1:(n2 - 1)) {
    AA[i * n1, i * n1 + 1] <- AA[i * n1 + 1, i * n1] <- 1
}
AA[1, n1 * n2] <- AA[n1 * n2, 1] <- 1
K50 <- graph_from_adjacency_matrix(AA, "undirected", diag = FALSE)

The figure below shows such a graph, consisting of 50 cliques of size 5. 

In [ ]:
ggraph(K50, "stress") +
    geom_edge_link0(edge_width = 0.6, edge_color = "grey66") +
    geom_node_point(shape = 21, fill = "grey66", size = 2, show.legend = FALSE) +
    theme_void() +
    coord_fixed()

Intuitively, any clustering method should return a cluster for each clique.

In [ ]:
clu_louvain <- cluster_louvain(K50)
table(membership(clu_louvain))

A clustering algorithm that fixes this issue is the leiden algorithm.

In [ ]:
clu_leiden <- cluster_leiden(K50, objective_function = "CPM", resolution_parameter = 0.5)
table(membership(clu_leiden))

The figure below shows the clusters computed with the louvain method in grey and the leiden method in red.

In [ ]:
V(K50)$louvain <- membership(clu_louvain)
V(K50)$leiden <- membership(clu_leiden)
ggraph(K50, "stress") +
    geom_edge_link0(edge_width = 0.6, edge_color = "grey66") +
    geom_node_point(shape = 21, fill = "grey66", size = 1.5, show.legend = FALSE) +
    ggforce::geom_mark_ellipse(aes(x, y, group = louvain), expand = unit(4.5, "pt"), fill = "black", alpha = 0.25) +
    ggforce::geom_mark_circle(aes(x, y, group = leiden), expand = unit(4.5, "pt"), col = "firebrick3", size = 1) +
    theme_void() +
    coord_fixed()

I'll spare the technical details of the leiden method. If you are interested, check out the [original paper](https://www.nature.com/articles/s41598-019-41695-z). 

---

# Blockmodeling

Blockmodeling is similar to 

(Stochastic) Blockmodels, for instance,
can also be used to find community structures. Several packages exist for this, such as `randnet` or `blockmodels`.

---

# Core-Periphery structures

---

# I want to learn about ...

##  two mode networks

A two mode network is a network that consists of two disjoint sets of nodes (like people and events). 
Ties connect the two sets, e. g. participation of people in events. There exists a great variety
of two mode networks. The most often encountered ones are 

- Affiliation networks (Membership in institutions)
- Voting/Sponsorship networks (politicians and bills)
- Citation network (authors and papers)
- Co-Authorship networks (authors and papers)

Below we will discuss some methods via the famous "southern women" dataset consisting of
18 women who attended a series of 14 events.

In [ ]:
data("southern_women")

In [ ]:
ggraph(southern_women, "stress") +
    geom_edge_link(edge_color = "grey66") +
    geom_node_point(aes(fill = type, shape = type), size = 8, show.legend = FALSE) +
    geom_node_text(aes(label = name)) +
    scale_shape_manual(values = c(21, 22)) +
    theme_void()

The adjacency matrix of a two mode network is also referred to as an incidence matrix and can be obtained via
`as_incidence_matrix()`

In [ ]:
A <- as_incidence_matrix(southern_women)
A

The `tnet` ([link](https://CRAN.R-project.org/package=tnet)) and `bipartite` ([link](https://CRAN.R-project.org/package=bipartite)) offer some methods to analyse two mode networks directly, by 
adapting tools for standard (one-mode) networks (like the ones described above).

Besides analyzing a two-mode network as-is, there is also the possibility to project it to one mode. 
Mathematically, this is done by calculating $AA^T$ or $A^TA$, depending which mode we project on.
As an example, consider the southern women dataset again.

In [ ]:
B <- A %*% t(A)
B

This matrix can now be interpreted as a weighted network among the 18 women. Each entry corresponds to the number of times
two women went to the same event.

In [ ]:
proj <- graph_from_adjacency_matrix(B, weighted = TRUE, diag = FALSE, mode = "undirected")

ggraph(proj, "stress") +
    geom_edge_link(aes(edge_width = weight), edge_color = "grey66", show.legend = FALSE) +
    geom_node_point(shape = 21, fill = "grey66", size = 8, show.legend = FALSE) +
    geom_node_text(aes(label = name)) +
    scale_edge_width(range = c(1, 4)) +
    theme_void() +
    coord_cartesian(clip = "off")

As you can see, the network has become very dense. A very common step is now to binarize the
network. In doing so, we basically turn the network into a simple undirected one-mode network. This
makes all methods we described in the first few sections applicable to the network (at least in theory).
The simplest way of binarizing a weighted projection is to define a global threshold and remove a tie if
its weight is below the global threshold. This is simple but come with many undesirable structural problems.
More sophisticated tools work with statistical models in the background which determine if an edge weight
differs enough from the expected value. If so, the edge is kept in the binary "backbone" of the network. 

All possible backbone extraction methods are implemented in the `backbone` package ([link](https://CRAN.R-project.org/package=backbone)). An introduction to the package can be
found on [arxiv](https://arxiv.org/abs/1912.12779).

##  signed networks

Traditional SNA usually deals with relations among entities (e.g. people) that are positive, including “friendship”, “advice seeking”, etc. Most network analytic tools are devised under this premise, be that centrality indices, clustering tools and so forth. But of course not all occurring relations are positive. People can be friends but also foes.

This gives rise to signed networks. These networks are usually composed of both, positive and negative, ties measured among a set of entities. Traditional network analytic tools are not applicable to such networks without adapting for the presents of negative ties. The `signnet` package ([link](https://CRAN.R-project.org/package=signnet)) brings together methods that have been developed to analyse signed networks. This includes

- Structural balance ([tutorial](http://signnet.schochastics.net/articles/structural_balance.html))
- Blockmodeling ([tutorial](http://signnet.schochastics.net/articles/blockmodeling.html))
- Centrality ([tutorial](http://signnet.schochastics.net/articles/centrality.html))
- Signed two-mode networks ([tutorial](http://signnet.schochastics.net/articles/signed_2mode.html))

A dedicated tutorial for each methodology is given in the package vignettes, also linked above. Below, we
just briefly discuss the structure of the package.

The foundation of `signnet` is provided by `igraph`. All functions in the package 
assume that an igraph object is a signed network if it has an edge attribute “sign” with values 1 (positive) or -1 (negative).

In [ ]:
library(signnet)
g <- graph.full(5, directed = FALSE, loops = FALSE)
E(g)$sign <- 1
g

All methods should throw an error if the sign attribute is missing or contains other values than -1 and 1.

Matrices associated with a signed network follow the `igraph` naming scheme. The signed adjacency matrix can be obtained with `as_adj_signed()`.

In [ ]:
data("tribes")
as_adj_signed(tribes)[1:5, 1:5]

The signed Laplacian matrix is obtained by `laplacian_matrix_signed()`.

In [ ]:
laplacian_matrix_signed(tribes)[1:5, 1:5]

A function not explicitly mentioned in the tutorials linked above is `triad_census_signed()` which calculates the
signed triad census of a directed signed network. While the unsigned triad census has only 16 possible outcomes, there are 138 non-isomorphic signed triads, shown below.
![](signed_triads.png)
The naming scheme is "xxx-yyyyyy" where "xxx" corresponds to the name of the respective unsigned triad and "yyyyyy" is 
a string of "0", "N", "P", describing the type of ties present. So "300-NNNNNN" is a triad with all ties present and all ties are negative. 

The package includes two well known datasets to play with.

The “tribes” dataset is a signed social network of tribes of the Gahuku–Gama alliance structure of the Eastern Central Highlands of New Guinea. The network contains sixteen tribes connected by friendship (“rova”) and enmity (“hina”).

The “cowList” dataset contains a list of 52 signed networks of inter-state relations over time (1946-1999). Two countries are connected by a positive tie if they form an alliance or have a peace treaty. A negative tie exists between countries who are at war or in other kinds of conflicts. The dataset is derrived from the [correlates of war](https://correlatesofwar.org/).

##  ego networks

If you want to analyze ego networks, then I can only recommend this
[book](http://www.raffaelevacca.com/egocentric-r-book/) by [Raffaele Vacca](http://www.raffaelevacca.com/).
Raffaele has given countless workshops on ego network analysis in R and his material should provide
you with everything you need.


##  multilevel networks


For analyzing multilevel networks, I recommend the `multinet` pakage ([link](https://CRAN.R-project.org/package=multinet)).
Check out this [JSS Paper](http://multilayer.it.uu.se/jss.pdf) for a brief introduction into the package.

If you just want to visualize a multilevel network, then head over to my [network visualization](../netvizr/#multilevel-networks) tutorial.


Multigraphs are network representations in which multiple edges and edge loops (self edges) are permitted.
In R, there are at least two relevant packages. The first is `multigraph` ([link](https://CRAN.R-project.org/package=multigraph)) which implements some visualization methods for 
multigraphs.

The second is `multigraphr` ([link](https://CRAN.R-project.org/package=multigraphr)) which comes with a
series of statistical methods to study local and global properties of such graphs and goodness of fit tests.
The [vignette](https://cran.r-project.org/web/packages/multigraphr/vignettes/multigraphr.html) of the package is
a brilliant starting point for using the package.

For the technical details behind `multigraphr`, you can refer to:

>Shafie, T. (2015). A multigraph approach to social network analysis. Journal of Social Structure, 16. ([link](https://www.exeley.com/journal_of_social_structure/doi/10.21307/joss-2019-011))

>Shafie, T. (2016). Analyzing local and global properties of multigraphs. The Journal of Mathematical Sociology, 40(4), 239-264. ([link](https://www.tandfonline.com/doi/abs/10.1080/0022250X.2016.1219732?journalCode=gmas20))

>Shafie, T. and Schoch, D., (2021). Multiplexity analysis of networks using multigraph representations. Statistical Methods & Applications 30 (5), 1425-1444 ([link](https://link.springer.com/article/10.1007/s10260-021-00596-0))

>Shafie, T. (Under review). Goodness of fit tests for random multigraph models.

##  something else

Cant find what you are looking for? Ping me on [twitter](https://twitter.com/schochastics) and I see what I can do.

---

# Loading network data into R

`igraph` can deal with many different foreign network formats with the function `read_graph`.
(The `rgexf` package can be used to import Gephi files.)


In [ ]:
read_graph(file, format = c(
    "edgelist", "pajek", "ncol", "lgl",
    "graphml", "dimacs", "graphdb", "gml", "dl"
), ...)

(*I personally have encountered some issue when importing `graphml` files that were 
produced in [visone](https://visone.ethz.ch/html/about.html). Also `dl` files sometimes through an error.*)

If your network data is in one of the above formats you will find it easy to import
your network. 

If your data is not in a network file format, you will need one of the following functions to turn raw network data into an `igraph` object:
`graph_from_edgelist()`, `graph_from_adjacency_matrix()`, `graph_from_adj_list()`, or
`graph_from_data_frame()`.

Before using these functions, however, you still need to get the raw data into R. The concrete procedure
depends on the file format. If your data is stored as an excel spreadsheet, you need additional packages.
If you are familiar with the `tidyverse`, you can use the `readxl` package. Other options are, e.g. the `xlsx` package.

Most network data you'll find is in a plain text format (csv or tsv), either as an edgelist or adjacency matrix.
To read in such data, you can use base R's `read.table()`.

Make sure you check the following before trying to load a file: Does it contain a header (e.g. row/column names of an adjacency matrix)? How are values delimited (comma, whitespace or tab)? This is important to set the parameters `header`, `sep` to read the data properly.


If you are struggling with this step, you can use the `Netreader` Rstudio add in which comes with the `snahelper` package

![](Netreader1.png)
Using the `Netreader` should comes with a learning effect (hopefully). The last tab shows the R code to produce the network with the chosen data without using the Addin.

![](Netreader2.png)

Since the addin is supposed to be very general, the code might look a bit awkward.
The functions `graph_from_edgelist()` and `graph_from_adjacency_matrix()` come with a drawback. You can't add node or edge attributes automatically. 

In general, it is always a good idea to organize network data in two different data frames. One for the nodes (with attributes)
and one for the edges (edgelist+attributes). This will save you a lot of pain when trying to convert the 
data into an igraph object. Especially if you want to go down the [tidy way](../tidynetanar/) later.



# What is missing?

I have tried to cover as many of the available packages and tools to analyze networks in R as possible 
and will include more details (and packages) over time. What I will probably never cover though are 
packages related to network dynamics, such as `ergm` or `Rsiena`. These are just not my `r emo::ji("tea")`. 
For `ergm`, you can refer to this [statnet tutorial](http://statnet.org/Workshops/ergm_tutorial.html) and for 
`Rsiena` you can check out the [Rsiena webpage](https://www.stats.ox.ac.uk/~snijders/siena/siena_scripts.htm).
